In [2]:
import cv2
import numpy as np

In [ ]:

def extract_frequency_components(img, mode, cutoff=40):
    """
    Extracts high or low frequency components from an image using a Gaussian filter.
    
    Args:
        img (ndarray): Input image.
        mode (str): 'low_freq' or 'high_freq'.
        cutoff (float): Determines the filter size.

    Returns:
        ndarray: Processed image with extracted frequency components.
    """
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    dft = np.fft.fft2(gray)
    dft_shift = np.fft.fftshift(dft)

    rows, cols = gray.shape
    crow, ccol = rows // 2, cols // 2

    # Gaussian filter
    x, y = np.ogrid[:rows, :cols]
    if mode == 'low_freq':
        mask = np.exp(-((x - crow)**2 + (y - ccol)**2) / (2.0 * cutoff**2))
    elif mode == 'high_freq':
        mask = 1 - np.exp(-((x - crow)**2 + (y - ccol)**2) / (2.0 * cutoff**2))
    else:
        raise ValueError("Mode must be 'low_freq' or 'high_freq'")

    # Apply mask to frequency domain
    filtered = dft_shift * mask

    # Transform back to the spatial domain
    img_back = np.fft.ifft2(np.fft.ifftshift(filtered))
    img_back = np.abs(img_back)

    # Normalize to 8-bit range
    img_back = cv2.normalize(img_back, None, 0, 255, cv2.NORM_MINMAX)
    return img_back.astype(np.uint8)

def blend_images(img1, img2, cutoff=40):
    """
    Blends high-frequency components of img1 with low-frequency components of img2 and vice versa.
    
    Args:
        img1 (ndarray): First input image.
        img2 (ndarray): Second input image.
        cutoff (float): Determines the filter size.

    Returns:
        tuple: Two blended images (high_on_low, low_on_high).
    """
    # Extract components
    low_freq_img1 = extract_frequency_components(img1, 'low_freq', cutoff)
    high_freq_img1 = extract_frequency_components(img1, 'high_freq', cutoff)
    low_freq_img2 = extract_frequency_components(img2, 'low_freq', cutoff)
    high_freq_img2 = extract_frequency_components(img2, 'high_freq', cutoff)

    # Blend components
    high_on_low = cv2.addWeighted(low_freq_img2, 0.5, high_freq_img1, 0.5, 0)
    low_on_high = cv2.addWeighted(low_freq_img1, 0.5, high_freq_img2, 0.5, 0)

    return high_on_low, low_on_high

def mishmashmosh(img_path1, img_path2, cutoff=40):
    """
    Main function to read images, process them, and display the results.
    
    Args:
        img_path1 (str): Path to the first image.
        img_path2 (str): Path to the second image.
        cutoff (float): Determines the filter size.
    """
    # Read images
    img1 = cv2.imread(img_path1)
    img2 = cv2.imread(img_path2)

    if img1 is None or img2 is None:
        print("Error: One of the image paths is invalid.")
        return

    # Blend images
    high_on_low, low_on_high = blend_images(img1, img2, cutoff)

    # Display results
    cv2.imshow('High on Low', high_on_low)
    cv2.imshow('Low on High', low_on_high)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [4]:
# Replace with paths to your images
img_path1 = "/home/oem/eliza/DL/project/data/artifact/real/a-y-jackson_st-john-s-newfoundland-1951.jpg"
img_path2 = "/home/oem/eliza/DL/project/data/artifact/generated/art_painting/img000245.jpg"

mishmashmosh(img_path1, img_path2, cutoff=40)

: 